In [3]:
import numpy as np
from pydrake.all import (AddDefaultVisualization, Binding, LinearConstraint,
                         MathematicalProgram, PointCloud, Rgba, RigidTransform,
                         RobotDiagramBuilder, RollPitchYaw,
                         SceneGraphCollisionChecker, Solve, StartMeshcat,
                         VisibilityGraph)

In [4]:
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7001


In [5]:
# A movable sphere with fixed boxes in all corners.
# ┌─────┬───┬─────┐
# │     │   │     │
# │     │   │     │
# ├─────┘   └─────┤
# │       o       │
# ├─────┐   ┌─────┤
# │     │   │     │
# │     │   │     │
# └─────┴───┴─────┘
boxes_in_corners = """
<robot name="boxes">
  <link name="fixed">
    <collision name="top_left">
      <origin rpy="0 0 0" xyz="-1 1 0"/>
      <geometry><box size="1 1 1"/></geometry>
    </collision>
    <collision name="top_right">
      <origin rpy="0 0 0" xyz="1 1 0"/>
      <geometry><box size="1 1 1"/></geometry>
    </collision>
    <collision name="bottom_left">
      <origin rpy="0 0 0" xyz="-1 -1 0"/>
      <geometry><box size="1 1 1"/></geometry>
    </collision>
    <collision name="bottom_right">
      <origin rpy="0 0 0" xyz="1 -1 0"/>
      <geometry><box size="1 1 1"/></geometry>
    </collision>
  </link>
  <joint name="fixed_link_weld" type="fixed">
    <parent link="world"/>
    <child link="fixed"/>
  </joint>
  <link name="movable">
    <collision name="sphere">
      <geometry><sphere radius="0.1"/></geometry>
    </collision>
  </link>
  <link name="for_joint"/>
  <joint name="x" type="prismatic">
    <axis xyz="1 0 0"/>
    <limit lower="-2" upper="2"/>
    <parent link="world"/>
    <child link="for_joint"/>
  </joint>
  <joint name="y" type="prismatic">
    <axis xyz="0 1 0"/>
    <limit lower="-2" upper="2"/>
    <parent link="for_joint"/>
    <child link="movable"/>
  </joint>
</robot>
"""

meshcat.Set2dRenderMode(X_WC=RigidTransform(RollPitchYaw(-np.pi / 2, 0, 0),
                                            [0, 0, -5]),
                        xmin=-2,
                        xmax=2,
                        ymin=-2,
                        ymax=2)

builder = RobotDiagramBuilder(0.0)
robot_model_instances = builder.parser().AddModelsFromString(
    boxes_in_corners, "urdf")
builder.plant().Finalize()
AddDefaultVisualization(builder.builder(), meshcat=meshcat)
distance_function_weights = [1] * builder.plant().num_positions()

checker = SceneGraphCollisionChecker(model=builder.Build(),
            robot_model_instances=robot_model_instances,
            distance_function_weights = distance_function_weights,
            edge_step_size=0.01)
checker.model().ForcedPublish(checker.model_context().model_context())
meshcat.SetProperty("/drake/proximity", "visible", True)

# Sample random points
rng = np.random.default_rng(0)
N = 1000
points = rng.uniform(-1.5, 1.5, size=(3, N))
points[2, :] = 0.7
cloud = PointCloud(N)
cloud.mutable_xyzs()[:] = points
meshcat.SetObject("points", cloud, point_size=0.02, rgba=Rgba(0, 0, 0, 1.0))
meshcat.SetTransform("points", RigidTransform())

indices = [
    i for i, val in enumerate(
        checker.CheckConfigsCollisionFree(points[:2, :].T, parallelize=True))
    if val
]
points = points[:2, indices]
A = VisibilityGraph(checker, points)


INFO:drake:Allocating contexts to support 1 parallel queries given omp_num_threads 1 omp_max_threads 1 and omp_thread_limit 1 OpenMP enabled in build? false


In [6]:
def max_clique(A):
    N = A.shape[0]
    prog = MathematicalProgram()
    v = prog.NewBinaryVariables(N, "v")
    prog.AddLinearCost(-np.ones(N), v)
    # Make the constraint once, and use it many times.
    c = LinearConstraint(np.ones((1,2)), [0],[1])  # [1, 1] x <= 1.
    for i in range(N):
        for j in range(i):
            if A[i,j] == 0:
                # v[i] + v[j] <= 1
                prog.AddConstraint(
                    binding=Binding[LinearConstraint](c, [v[i], v[j]]))

    result = Solve(prog)
    return [i for i, val in enumerate(result.GetSolution(v)) if val]


In [8]:
def clique_cover(A, min_clique_size=3):
    indices = range(A.shape[0])
    while True:
        clique = max_clique(A)
        if (len(clique) < min_clique_size):
            break
        print(clique)
        mask = np.ones(A.shape[0], dtype=bool)
        mask[clique] = False
        indices = np.delete(indices, clique)
        A = A[:, mask]
        A = A[mask, :]

clique_cover(A)


[3, 4, 6, 7, 9, 10, 14, 16, 17, 20, 21, 22, 24, 28, 29, 30, 33, 34, 35, 36, 37, 38, 40, 41, 42, 45, 47, 48, 50, 52, 55, 57, 59, 60, 61, 62, 64, 65, 66, 67, 68, 71, 78, 79, 80, 82, 85, 86, 87, 88, 89, 90, 92, 94, 95, 96, 98, 100, 101, 102, 105, 106, 108, 110, 113, 115, 116, 117, 118, 120, 121, 122, 125, 127, 128, 130, 132, 133, 134, 135, 136, 138, 140, 141, 144, 145, 146, 149, 150, 152, 155, 157, 159, 160, 161, 163, 169, 170, 171, 173, 174, 175, 176, 178, 179, 183, 184, 187, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 206, 207, 208, 209, 210, 211, 213, 215, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 228, 229, 231, 233, 234, 236, 238, 240, 241, 242, 243, 245, 246, 247, 248, 250, 252, 253, 254, 255, 257, 259, 260, 264, 265, 266, 268, 270, 271, 272, 273, 278, 280, 281, 283, 284, 285, 287, 293, 294, 296, 297, 299, 300, 301, 303, 304, 306, 307, 309, 311, 314, 316, 317, 320, 321, 323, 324, 327, 328, 330, 331, 334, 335, 336, 337, 338, 340, 341, 342, 343, 347, 3

In [ ]:
# Deleting columns with indices 1 and 2
#A = np.delete(A, [1, 2], axis=0)
print(A.shape)